#**Task-1**
Import libraries and load dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
movies=pd.read_csv("/content/movies.csv")
ratings=pd.read_csv("/content/ratings.csv")

movies.head()

In [ ]:
ratings.head()

#Task-2
Exploratory Data Analysis including:

---> Understanding of distribution of the features available

---> Finding unique users and movies

---> Average rating and Total movies at genre level.

---> Unique genres considered..

In [ ]:
movies.info()

In [ ]:
ratings.info()

In [ ]:
movies.describe()

In [ ]:
ratings.describe()

In [ ]:
movies.shape

In [ ]:
ratings.shape

In [ ]:
# Understanding of distribution of the features available
movies.columns

In [ ]:
ratings.columns

In [ ]:
movies.isnull().sum()

In [ ]:
ratings.isnull().sum()

In [ ]:
movies.isnull().any()

In [ ]:
ratings.isnull().any()

In [ ]:
movies.duplicated()

In [ ]:
ratings.duplicated()

In [ ]:
# Finding unique users and movies
movies["title"].unique()

In [ ]:
ratings["userId"].unique()

In [ ]:
df=pd.merge(movies,ratings,how="left",on="movieId")
df.head()

In [ ]:
#Average rating and Total movies at genre level
average_ratings = df.groupby('genres')['rating'].mean()
total_movies = df.groupby('genres')['movieId'].count()

In [ ]:
average_ratings

In [ ]:
total_movies

In [ ]:
# Unique genres considered
df["genres"].unique()

## 1. Create a popularity-based recommender system at a genre level. The user will input a genre (g), minimum rating threshold (t) for a movie, and no. of recommendations(N) for which it should be recommended top N movies which are most popular within that genre (g) ordered by ratings in descending order where each movie has at least (t) reviews.

In [ ]:
## Removing year from the title for better interpretation
remove=df['title'].str[:-7].reset_index()
remove.head()

In [ ]:
df['title']=remove['title']
df.head()

In [ ]:
#Calculate mean rating of all movies
ratings_mean=df.groupby(['title','genres'])['rating'].mean().sort_values(ascending=False)
ratings_mean

In [ ]:
#Calculate count rating of all movies
ratings_count=df.groupby(['title','genres'])['rating'].count().sort_values(ascending=False)
ratings_count

In [ ]:
#Creating dataframe with 'rating' count values
ratings = pd.DataFrame(ratings_mean)
ratings.head(10)

In [ ]:
ratings['num of ratings'] = pd.DataFrame(ratings_count)
ratings.head(10)

In [ ]:
popular_movies=ratings.sort_values('num of ratings',ascending=False).reset_index()
popular_movies.head(10)

In [ ]:
def min_reviews_threshold_data(genre,min_reviews_threshold,No_of_recommendations):
    genres_data=popular_movies[popular_movies['genres']=='Comedy'].reset_index()
    pre_final_data=genres_data[genres_data['num of ratings']>100]
    data=pre_final_data.drop(columns=['index','genres'])[:No_of_recommendations]
    final_data=data.rename(columns={'title':'Movie Title','rating':'Average Movie Rating Num','num of ratings':'Reviews'})
    return final_data

In [ ]:
min_reviews_threshold_data('Comedy',100,5)

## 2. Create a content-based recommender system that recommends top N movies based on similar movie(m) genres.

In [ ]:
df.head()

In [ ]:
movie='Toy Story'
genre=df[df.title==movie]
genre.head(10)

In [ ]:
genre=df[df.title=='Toy Story']
unique_genre=genre.genres.unique()[0]
unique_genre

'Adventure|Animation|Children|Comedy|Fantasy'

In [ ]:
genres_based=df[df.genres==unique_genre]
genres_based

In [ ]:
new=genres_based.groupby(['title','genres'])['movieId'].count().reset_index().sort_values('movieId',ascending=False)
abc_set_index=new.set_index('title')
abc_set_index

In [ ]:
abc_set_index.drop(['Toy Story'],axis=0,inplace=True)
abc_set_index[:10]

In [ ]:
def content_based_filtering(movie_title,no_of_recos):
    check_genre=df[df.title==movie_title]
    unique_genre=check_genre.genres.unique()[0]
    new=genres_based.groupby(['title','genres'])['movieId'].count().reset_index().sort_values('movieId',ascending=False)
    abc_set_index=new.set_index('title')
    abc_set_index.drop([movie_title],axis=0,inplace=True)
    mask_indices=abc_set_index.drop(columns=['movieId','genres']).reset_index()[:no_of_recos]
    return mask_indices

In [ ]:
content_based_filtering('Shrek the Third',5)

## 3. Create a collaborative based recommender system which recommends top N movies based on “K” similar users

In [ ]:
vc=df.movieId.value_counts()  #Based on movie id, how many times the user id's repeated... ratings given
vc

In [ ]:
ac=df.userId.value_counts()  #Based on user id, how many times the user id's repeated... ratings given
ac

In [ ]:
vc[vc>50]

In [ ]:
popular_movies=vc[vc>50].index   #Here Iam taking 50 for popular movies
popular_movies

In [ ]:
ac.median()

In [ ]:
prolific_users=ac[ac>200].index                     #Here Iam taking 300 for popular/prolific users
prolific_users=prolific_users.astype("int64")
prolific_users

In [ ]:
df[df.userId.isin(prolific_users)] #checking overall merged_dataset with prolific users

In [ ]:
final_ratings=df[df.userId.isin(prolific_users) & df.movieId.isin(popular_movies)]
final_ratings.tail()

In [ ]:
new=genres_based.groupby(['title','genres'])['movieId'].count().reset_index().sort_values('movieId',ascending=False)
abc_set_index=new.set_index('title')
abc_set_index

In [ ]:
df=new.drop(["movieId"],axis=1)
df

##Additional/Optional: Create a GUI interface using Python libraries (ipywidgets etc.) to play around with the recommendation modules

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Sample recommendation function
def recommend_movies(genre, rating):
    # Replace this with your recommendation logic
    recommendations = df['title']
    return recommendations

# GUI function
def recommend_movies_gui():
    genre_dropdown = widgets.Dropdown(
        options=['Action', 'Drama', 'Comedy', 'Sci-Fi'],
        description='Genre:'
    )
    rating_slider = widgets.IntSlider(
        value=5,
        min=1,
        max=10,
        step=1,
        description='Minimum Rating:'
    )
    recommend_button = widgets.Button(
        description='Recommend Movies'
    )
    output = widgets.Output()

    def on_recommend_button_clicked(b):
        with output:
            output.clear_output()
            recommendations = recommend_movies(genre_dropdown.value, rating_slider.value)
            print("Recommended Movies:")
            for movie in recommendations:
              print("-", movie)

    recommend_button.on_click(on_recommend_button_clicked)

    display(widgets.VBox([genre_dropdown, rating_slider, recommend_button, output]))

# Run the GUI
recommend_movies_gui()